#TODO: get generals working, primary 14 elections working

In [177]:
import pandas as pd
import json
import numpy as np
import geopandas as gpd
import os
from bs4 import BeautifulSoup
import requests
import json

In [12]:
cd ../../data/results

[Errno 2] No such file or directory: 'results'
/home/kirk/centristDataScience/data/results


In [178]:
ls

CO_2004.tab*                  co_2016_general.json
CO_2006.tab*                  co_2016_general_turnout.json
CO_2008.tab*                  co_2016_primary.json
CO_2010.tab*                  co_2016_primary_turnout.json
co_2012_general.json          colorado_1968_2010.csv
co_2012_primary.json          colorado_specific_columns.csv
co_2014_general.json          icpsr_clean.csv
co_2014_general_turnout.json  primary_data.json


In [179]:
dv_files = filter(lambda x: x.endswith('tab'), os.listdir('.'))
dv_dfs = map(lambda x: pd.read_table(x), dv_files)
icpsr = pd.read_csv('colorado_specific_columns.csv')
icpsr = icpsr[icpsr.columns[1:]]

In [180]:
for i in dv_dfs:
    print i.columns

Index([u'state', u'year', u'county', u'precinct', u'g2006_GOV_dv', u'g2006_GOV_rv', u'g2006_USH_dv', u'g2006_USH_rv'], dtype='object')
Index([u'state', u'year', u'vtd', u'g2008_USP_dv', u'g2008_USP_rv', u'g2008_USS_dv', u'g2008_USS_rv', u'g2008_USH_dv', u'g2008_USH_rv', u'g2008_SBOE_dv', u'g2008_SBOE_rv', u'g2008_STH_rv', u'g2008_STH_dv', u'g2008_STS_rv', u'g2008_STS_dv', u'g2008_REG_dv', u'g2008_REG_rv', u'g2008_RGNT_dv', u'g2008_RGNT_rv'], dtype='object')
Index([u'state', u'year', u'vtd', u'g2010_USS_rv', u'g2010_USS_dv', u'g2010_GOV_rv', u'g2010_GOV_dv', u'g2010_SOS_rv', u'g2010_SOS_dv', u'g2010_ATG_rv', u'g2010_ATG_dv', u'g2010_TRE_rv', u'g2010_TRE_dv', u'g2010_USH_rv', u'g2010_USH_dv', u'g2010_SBOE_rv', u'g2010_SBOE_dv', u'g2010_STS_rv', u'g2010_STS_dv', u'g2010_STH_rv', u'g2010_STH_dv', u'g2010_REG_rv', u'g2010_REG_dv'], dtype='object')
Index([u'state', u'year', u'county', u'precinct', u'cd', u'g2004_USP_dv', u'g2004_USP_rv', u'g2004_USS_dv', u'g2004_USS_rv', u'g2004_USH_dv', u'g

In [181]:
new_cols=['id', 'state', 'year', 'month', 'senate_or_house', 'district_name', 'district_number', 'election_type',
         'determines_sitting_leg', 'candidate_id', 'candidate_name', 'detailed_party', 'simple_party', 'incumbent', 
         'candidate_total', 'winner', 'num_candidates', 'num_dem', 'num_repub', 'num_other', 'total_votes', 
          'total_dem_votes','total_repub_votes', 'total_other_votes', 'highest_percentage', 'second_highest_percentage',
         'percent_MoV', 'candidate_percent', 'candidate_winner_dif']

In [182]:
icpsr.columns = new_cols

In [183]:
dt = lambda x: pd.to_datetime(str(x['year']) + '/' + str(x['month']))
e = icpsr.apply(dt, 1)
icpsr['election_date'] = e

In [184]:
c = filter(lambda x: x != 'district_name', icpsr.columns)
icpsr = icpsr[c]

In [185]:
icpsr.simple_party.replace([200, 100, 400, 600], ['repub', 'dem', 'indep', 'write-in'], True)

In [186]:
icpsr.to_csv('icpsr_clean.csv')

In [151]:
def matchDistrictsCounties(url):
    r = requests.get(url)
    s = BeautifulSoup(r.content)
    districts = map(lambda x: x.text, s.findAll('p', {'class':'pagehead5'}))
    dist = map(lambda x: {'district': x[0], 'counties': x[1]}, 
               zip(districts, map(lambda x: filter(lambda g: g != 'Total',
                                                   map(lambda y: y.text, x.findAll('strong')[:-3])),
                                  s.findAll('table')[1:-1])))
    return dist

def getTurnoutDataFrame(url):
    r = requests.get(url)
    s = BeautifulSoup(r.content)
    rows = s.findAll('table')[1].findAll('tr')[1:-1]
    d = pd.DataFrame(map(lambda f: {'county':f[0], 'turnout':float(f[-1][:-1])}, 
                         map(lambda x: map(lambda g: g[0], map(lambda y: list(y.stripped_strings),
                                                               x.findAll('td'))),
                             rows)))
    return d

def findDistrictAverageTurnout(dist_dict, turnout_df):
    d1 = map(lambda x: dict(x.items() + [('turnout_average', turnout_df[turnout_df['county'].isin(x['counties'])].turnout.mean())]), 
             dist_dict)
    return d1

def cleanElection(results_url, turnout_url):
    dists = matchDistrictsCounties(results_url)
    df = getTurnoutDataFrame(turnout_url)
    res = findDistrictAverageTurnout(dists, df)
    return res

In [168]:
primary_12_house = cleanElection('https://www.sos.state.co.us/pubs/elections/Results/Abstract/2012/primary/republican/stateReps.html',
                                'https://www.sos.state.co.us/pubs/elections/Results/Abstract/2012/primary/turnout.html')
primary_12_senate = cleanElection('https://www.sos.state.co.us/pubs/elections/Results/Abstract/2012/primary/republican/stateSenate.html',
                                 'https://www.sos.state.co.us/pubs/elections/Results/Abstract/2012/primary/turnout.html')

In [176]:
json.dump({'primary_12_house': primary_12_house, 'primary_12_senate':primary_12_senate},open('primary_data.json', 'w'))

In [169]:
primary_14_house

[]

In [170]:
matchDistrictsCounties('https://www.sos.state.co.us/pubs/elections/Results/Abstract/2014/primary/republican/stateSenate.html')

[]

In [174]:
icpsr.tail()

,id,state,year,month,senate_or_house,district_number,election_type,determines_sitting_leg,candidate_id,candidate_name,...,total_votes,total_dem_votes,total_repub_votes,total_other_votes,highest_percentage,second_highest_percentage,percent_MoV,candidate_percent,candidate_winner_dif,election_date
3503,296035,6,2010,11,9,62,G,1,283670,"VIGIL, EDWARD",...,21166,11842,9324,0,55.948219,44.051781,11.896439,55.948219,0.000000,2010-11-14
3504,296036,6,2010,11,9,63,G,1,296036,"BECKER, JON",...,19138,0,19138,0,100.000000,0.000000,100.000000,100.000000,0.000000,2010-11-14
3505,296037,6,2010,11,9,64,G,1,296037,"KELLOGG, LISA GRACE",...,21686,11144,10542,0,51.387993,48.612007,2.775986,48.612007,2.775986,2010-11-14
3506,296038,6,2010,11,9,64,G,1,20130,"MCKINLEY, WES",...,21686,11144,10542,0,51.387993,48.612007,2.775986,51.387993,0.000000,2010-11-14
3507,296039,6,2010,11,9,65,G,1,271478,"SONNENBERG, JERRY",...,21951,0,21951,0,100.000000,0.000000,100.000000,100.000000,0.000000,2010-11-14
